In [44]:
import requests
import pandas as pd
import io

In [47]:
def crawler(data_time):
    url =  'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + data_time + '&type=ALLBUT0999'
    page = requests.get(url).text
    use_text = page.splitlines()
    for i, text in enumerate(use_text):
        if text == '"證券代號","證券名稱","成交股數","成交筆數","成交金額","開盤價","最高價","最低價","收盤價","漲跌(+/-)","漲跌價差","最後揭示買價","最後揭示買量","最後揭示賣價","最後揭示賣量","本益比",':
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO((''.join([text[:-1] + '\n' for text in use_text[initial_point:]]))))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('"', ''))
    test_df['證券代號'] = test_df['證券代號'].apply(lambda x:x.replace('=', ''))
    return test_df

In [69]:
import datetime
import time

def trans_date(date_time):
    return str(date_time).split(' ')[0].replace('-', '')

def parse_n_days(start_date, n):
    df_dict = {}
    now_date = start_date
    for i in range(n):
        time.sleep(3)
        try:
            df = crawler(trans_date(now_date))
            print('Successful!!' + ' ' + trans_date(now_date))
            df_dict.update({trans_date(now_date):df})
        except:
            print('Fails at' + ' ' + trans_date(now_date))
        now_date = now_date - datetime.timedelta(days=1)
    return df_dict

In [70]:
result_dict = parse_n_days(datetime.datetime.now(), 6)

Successful!! 20220802
Successful!! 20220801
Fails at 20220731
Fails at 20220730
Successful!! 20220729
Successful!! 20220728


In [75]:
for key in result_dict.keys():
    result_dict[key].to_csv(str(key) + '.cvs')

In [78]:
stock_df = result_dict[key]

In [79]:
stock_df

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"8,557,891","8,632","1,004,795,134",117.85,117.95,116.90,116.90,+,0.20,116.90,51,116.95,1,0.00
1,0051,元大中型100,"22,082",169,"1,163,279",53.00,53.00,52.40,52.50,-,0.35,52.50,1,53.00,37,0.00
2,0052,富邦科技,"263,506",338,"26,145,040",100.00,100.00,98.70,98.70,-,0.20,98.70,6,98.95,11,0.00
3,0053,元大電子,"6,118",992,"349,286",57.85,57.85,57.00,57.20,+,0.60,57.00,1,57.35,1,0.00
4,0055,元大MSCI金融,"193,144",346,"4,365,968",22.52,22.67,22.52,22.64,+,0.27,22.64,1,22.65,10,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,9944,新麗,"68,219",61,"1,348,929",19.75,19.80,19.70,19.80,+,0.05,19.75,1,19.80,1,3.96
1167,9945,潤泰新,"6,584,802","5,418","400,025,261",61.00,61.50,60.30,60.80,,0.00,60.70,121,60.80,86,9.13
1168,9946,三發地產,"39,068",38,"502,029",13.00,13.00,12.80,12.90,-,0.05,12.90,1,13.00,6,18.43
1169,9955,佳龍,"217,582",223,"4,141,776",19.35,19.35,18.85,18.95,-,0.05,18.90,2,18.95,6,0.00
